In [1]:
import sys
sys.path.insert(0, r"K:/Thesis/codes/crypto_project")
sys.path.insert(0, r"E:/Thesis/crypto_project")
import os
import sqlite3
import pandas as pd
import numpy as np
from tqdm import tqdm

DATABASE_LOCATION = r"E:/Thesis/database"
from factor_model.model_update.database_generators import (
    RAW_DATA_DB,
    RETURN_DB,
    FACTOR_MODEL_ESTIMATES
)
from typing import Dict


In [2]:
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "database_server.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django

django.setup()

symbol = "BTC-USD"



In [3]:
from crypto_calculator.models import RawPriceData


raw_price_data = RawPriceData.objects.filter(symbol=symbol).values("close", "date")
df = pd.DataFrame(list(raw_price_data))
if len(list(raw_price_data)) > 0:
    df["date"] = df["date"].apply(lambda x: x.strftime("%Y-%m-%d"))
    df.set_index("date", inplace=True, drop=True)

In [4]:
len(df)

3459

In [8]:
df = df.diff()

In [13]:
df.pow(2).ewm(halflife=365, min_periods=20).mean()

,close
date,
2014-09-17,NaN
2014-09-18,NaN
2014-09-19,NaN
2014-09-20,NaN
2014-09-21,NaN
...,...
2024-03-02,1.009332e+06
2024-03-03,1.009872e+06
2024-03-04,1.058600e+06


In [12]:
df.pow(2).ewm(halflife=365, min_periods=20).mean().pow(0.5).dropna()

,close
date,
2014-09-17,NaN
2014-09-18,NaN
2014-09-19,NaN
2014-09-20,NaN
2014-09-21,NaN
...,...
2024-03-02,1004.654988
2024-03-03,1004.924084
2024-03-04,1028.882770


In [5]:
def create_ewma_std_estimates(df: pd.DataFrame, halflife: float=365, min_periods: float=20) -> pd.DataFrame:
    return df.ewm(halflife=halflife, min_periods=min_periods).std().dropna()

create_ewma_std_estimates(df).to_dict()

{'close': {'2014-10-06': 35.61878459112293,
  '2014-10-07': 36.63784262234197,
  '2014-10-08': 36.48143987833895,
  '2014-10-09': 35.88336563126529,
  '2014-10-10': 35.395634359225355,
  '2014-10-11': 34.898791209201164,
  '2014-10-12': 34.18706424971514,
  '2014-10-13': 33.54257276929729,
  '2014-10-14': 33.08096130102165,
  '2014-10-15': 32.538936506340086,
  '2014-10-16': 31.95859868200554,
  '2014-10-17': 31.406389679466184,
  '2014-10-18': 30.91114249121744,
  '2014-10-19': 30.425173275417475,
  '2014-10-20': 29.94724565166523,
  '2014-10-21': 29.492016525230316,
  '2014-10-22': 29.054297511929096,
  '2014-10-23': 28.95724979239346,
  '2014-10-24': 28.849760406814063,
  '2014-10-25': 29.037534209667175,
  '2014-10-26': 28.98097729396599,
  '2014-10-27': 28.951796038730343,
  '2014-10-28': 28.807480910428072,
  '2014-10-29': 29.270608591624566,
  '2014-10-30': 29.36996834582119,
  '2014-10-31': 29.64218717747635,
  '2014-11-01': 30.299666464426732,
  '2014-11-02': 30.86826182114373

In [8]:
def create_ewma_std_estimates(df: pd.DataFrame, halflife: float=365, min_periods: float=20) -> pd.DataFrame:
    return df.ewm(halflife=halflife, min_periods=min_periods).std().dropna()

In [5]:
from crypto_calculator.models import FactorReturns

return_data = FactorReturns.objects.values("market", "date")
df = pd.DataFrame(list(return_data))
df["date"] = df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
df.set_index("date", inplace=True, drop=True)

OperationalError: no such table: factor_returns

In [6]:
symbol = "BTC-USD"

with sqlite3.connect(
    os.path.join(DATABASE_LOCATION, RETURN_DB)
) as conn:
    return_df = pd.read_sql_query(
        f"SELECT * FROM returns where symbol = '{symbol}'",
        conn,
    )


In [3]:
with sqlite3.connect(
    os.path.join(DATABASE_LOCATION, RETURN_DB)
) as conn:
    return_df = pd.read_sql_query(
        f"SELECT * FROM returns",
        conn,
    )


In [ ]:
return_df

In [11]:
kk = pd.DataFrame({
    "symbol": list(return_df[return_df["date"] =="2024-09-07"]["symbol"].unique())[0:200]})
kk["value"] = 1
kk.to_csv("E:\Thesis\sample_input\port_200b.csv", index=False, header=False)

In [12]:
kk

,symbol,value
0,0X0-USD,1
1,0XDEV-USD,1
2,1000SATS-USD,1
3,10SET-USD,1
4,1ART-USD,1
...,...,...
195,AVINOC-USD,1
196,AVL-USD,1
197,AVT-USD,1
198,AWC-USD,1


In [9]:
return_df.tail()

,id,date,return,risk_free_rate,total_return,symbol
3124573,173,2024-09-04,0.015177,5.35,0.015326,ZYPTOTOKEN-USD
3124574,174,2024-09-05,-0.049600,5.35,-0.049451,ZYPTOTOKEN-USD
3124575,175,2024-09-06,-0.098685,5.35,-0.098536,ZYPTOTOKEN-USD
3124576,176,2024-09-07,0.076951,5.35,0.077100,ZYPTOTOKEN-USD
3124577,177,2024-09-08,0.018538,5.35,0.018687,ZYPTOTOKEN-USD
